In [2]:
import argparse
import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties

In [3]:
def Affine_Fit( from_pts, to_pts ):
    """Fit an affine transformation to given point sets.
      More precisely: solve (least squares fit) matrix 'A'and 't' from
      'p ~= A*q+t', given vectors 'p' and 'q'.
      Works with arbitrary dimensional vectors (2d, 3d, 4d...).

      Written by Jarno Elonen <elonen@iki.fi> in 2007.
      Placed in Public Domain.

      Based on paper "Fitting affine and orthogonal transformations
      between two sets of points, by Helmuth Späth (2003)."""

    q = from_pts
    p = to_pts
    if len(q) != len(p) or len(q)<1:
        print("from_pts and to_pts must be of same size.")
        return false

    dim = len(q[0]) # num of dimensions
    if len(q) < dim:
        print("Too few points => under-determined system.")
        return false

    # Make an empty (dim) x (dim+1) matrix and fill it
    c = [[0.0 for a in range(dim)] for i in range(dim+1)]
    for j in range(dim):
        for k in range(dim+1):
            for i in range(len(q)):
                qt = list(q[i]) + [1]
                c[k][j] += qt[k] * p[i][j]

    # Make an empty (dim+1) x (dim+1) matrix and fill it
    Q = [[0.0 for a in range(dim)] + [0] for i in range(dim+1)]
    for qi in q:
        qt = list(qi) + [1]
        for i in range(dim+1):
            for j in range(dim+1):
                Q[i][j] += qt[i] * qt[j]

    # Ultra simple linear system solver. Replace this if you need speed.
    def gauss_jordan(m, eps = 1.0/(10**10)):
      """Puts given matrix (2D array) into the Reduced Row Echelon Form.
         Returns True if successful, False if 'm' is singular.
         NOTE: make sure all the matrix items support fractions! Int matrix will NOT work!
         Written by Jarno Elonen in April 2005, released into Public Domain"""
      (h, w) = (len(m), len(m[0]))
      for y in range(0,h):
        maxrow = y
        for y2 in range(y+1, h):    # Find max pivot
          if abs(m[y2][y]) > abs(m[maxrow][y]):
            maxrow = y2
        (m[y], m[maxrow]) = (m[maxrow], m[y])
        if abs(m[y][y]) <= eps:     # Singular?
          return False
        for y2 in range(y+1, h):    # Eliminate column y
          c = m[y2][y] / m[y][y]
          for x in range(y, w):
            m[y2][x] -= m[y][x] * c
      for y in range(h-1, 0-1, -1): # Backsubstitute
        c  = m[y][y]
        for y2 in range(0,y):
          for x in range(w-1, y-1, -1):
            m[y2][x] -=  m[y][x] * m[y2][y] / c
        m[y][y] /= c
        for x in range(h, w):       # Normalize row y
          m[y][x] /= c
      return True

    # Augement Q with c and solve Q * a' = c by Gauss-Jordan
    M = [ Q[i] + c[i] for i in range(dim+1)]
    if not gauss_jordan(M):
        print("Error: singular matrix. Points are probably coplanar.")
        return false

    # Make a result object
    class Transformation:
        """Result object that represents the transformation
           from affine fitter."""

        def To_Str(self):
            res = ""
            for j in range(dim):
                str = "x%d' = " % j
                for i in range(dim):
                    str +="x%d * %f + " % (i, M[i][j+dim+1])
                str += "%f" % M[dim][j+dim+1]
                res += str + "\n"
            return res

        def Transform(self, pt):
            res = [0.0 for a in range(dim)]
            for j in range(dim):
                for i in range(dim):
                    res[j] += pt[i] * M[i][j+dim+1]
                res[j] += M[dim][j+dim+1]
            return res
    return Transformation()


In [4]:
def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R@centroid_A + centroid_B

    return R, t


In [5]:
animal = 'MD589'
fileLocationManager = FileLocationManager(animal)

atlas_name = 'atlasV7'
DATA_PATH = '/net/birdstore/Active_Atlas_Data/data_root'
ROOT_DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data'
THUMBNAIL_DIR = os.path.join(ROOT_DIR, animal, 'preps', 'CH1', 'thumbnail')
ATLAS_PATH = os.path.join(DATA_PATH, 'atlas_data', atlas_name)
ORIGIN_PATH = os.path.join(ATLAS_PATH, 'origin')
VOLUME_PATH = os.path.join(ATLAS_PATH, 'structure')
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')
#if os.path.exists(OUTPUT_DIR):
#    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)
origin_files = sorted(os.listdir(ORIGIN_PATH))
volume_files = sorted(os.listdir(VOLUME_PATH))
sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
surface_threshold = 0.8
SCALE = (10 / 0.46)

structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    structure = os.path.splitext(volume_filename)[0]
    if structure not in origin_filename:
        print(structure, origin_filename)
        break

    color = get_structure_number(structure.replace('_L', '').replace('_R', ''))

    origin = np.loadtxt(os.path.join(ORIGIN_PATH, origin_filename))
    volume = np.load(os.path.join(VOLUME_PATH, volume_filename))

    volume = np.rot90(volume, axes=(0, 1))
    volume = np.flip(volume, axis=0)
    volume[volume > surface_threshold] = color
    volume = volume.astype(np.uint8)

    structure_volume_origin[structure] = (volume, origin)

col_length = sqlController.scan_run.width/SCALE
row_length = sqlController.scan_run.height/SCALE
z_length = len(os.listdir(THUMBNAIL_DIR))
atlasV7_volume = np.zeros(( int(row_length), int(col_length), z_length), dtype=np.uint8)
print('atlas volume shape', atlasV7_volume.shape)

DK52_centers = {'12N': [46488, 18778, 242],
                '5N_L': [38990, 20019, 172],
                '5N_R': [39184, 19027, 315],
                '7N_L': [42425, 23190, 166],
                '7N_R': [42286, 22901, 291]}
##### actual data for both sets of points, pixel coordinates
MD589_centers = {'10N_L': [31002.069009677187, 17139.273764067697, 210],
                 '10N_R': [30851.821452912456, 17026.27799914138, 242],
                 '4N_L': [25238.351916435207, 13605.972626040299, 210],
                 '4N_R': [25231.77274616, 13572.152382002621, 236],
                 '5N_L': [25863.93885802854, 16448.49802904827, 160],
                 '5N_R': [25617.920248719453, 16089.048882550318, 298],
                 '7N_L': [27315.217906796195, 18976.4921239128, 174],
                 '7N_R': [27227.134448911638, 18547.6538128018, 296],
                 '7n_L': [26920.538205417844, 16996.292850204114, 177],
                 '7n_R': [26803.347723222105, 16688.23325135847, 284],
                 'Amb_L': [29042.974021303286, 18890.218579368557, 167],
                 'Amb_R': [28901.503217056554, 18291.072163747285, 296],
                 'DC_L': [28764.5378815116, 15560.1247992853, 134],
                 'DC_R': [28519.240424058273, 14960.063579837733, 330],
                 'LC_L': [26993.749068166835, 15146.987356709138, 180],
                 'LC_R': [26951.610128773387, 14929.363532303963, 268],
                 'Pn_L': [23019.18002537938, 17948.490571838032, 200],
                 'Pn_R': [23067.16403704933, 17945.89008778571, 270],
                 'SC': [24976.373217129738, 10136.880464106176, 220],
                 'Tz_L': [25210.29041867189, 18857.20817842522, 212],
                 'Tz_R': [25142.520897455783, 18757.457820947686, 262]}
centers = OrderedDict(MD589_centers)
centers_list = []
for value in centers.values():
    centers_list.append((value[1]/SCALE, value[0]/SCALE, value[2]))
COM = np.array(centers_list)
atlas_com_centers = OrderedDict()
atlas_all_centers = {}
for structure, (volume, origin) in sorted(structure_volume_origin.items()):
    midcol, midrow, midz = origin
    row_start = midrow + row_length / 2
    col_start = midcol + col_length / 2
    z_start = midz / 2 + z_length / 2
    row_end = row_start + volume.shape[0]
    col_end = col_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) / 2
    midcol = (col_end + col_start) / 2
    midrow = (row_end + row_start) / 2
    midz = (z_end + z_start) / 2
    if structure in centers.keys():
        atlas_com_centers[structure] = [midrow, midcol, midz]
    atlas_all_centers[structure] = [midrow, midcol, midz]
ATLAS_centers = OrderedDict(atlas_com_centers)
ATLAS = np.array(list(ATLAS_centers.values()))
#####Steps
#trn = Affine_Fit(ATLAS, COM)
R, t = rigid_transform_3D(ATLAS.T, COM.T)
pprint(R)
t = np.reshape(t, (1,3))
pprint(t)

atlas volume shape (1490, 2010, 447)
array([[ 0.99669058,  0.04718899, -0.06618979],
       [-0.04692206,  0.99888295,  0.0055824 ],
       [ 0.06637928, -0.00245817,  0.99779144]])
array([[-50.65916898, 276.72687969, -43.12235302]])


In [10]:
for structure, (volume, origin) in sorted(structure_volume_origin.items()):
    print(str(structure).ljust(7),end=": ")

    # transformed atlas
    arr = np.array(atlas_all_centers[structure])
    #results = trn.Transform(arr)
    input = arr + t
    results = np.dot(R, input.T)
    midrow = results[0][0]
    midcol = results[1][0]
    midz = results[2][0]
    row_start = int(round(midrow - volume.shape[1] / 2))
    col_start = int(round(midcol - volume.shape[0] / 2))
    sec_start = int(round(midz - (volume.shape[2] / 2) / 2))
    row_end = row_start + volume.shape[1]
    col_end = col_start + volume.shape[0]
    sec_end = int(round(sec_start + (volume.shape[2] + 1) // 2))

    ## generic atlas
    x, y, z = origin
    x_start = int( round(x + col_length / 2))
    y_start = int( round(y + row_length / 2))
    z_start = int(z) // 2 + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) // 2

    print('Row',
          str(y_start).rjust(4),
          str(y_end).rjust(4),
          'col',
          str(x_start).rjust(4),
          str(x_end).rjust(4),
          'z',
          str(z_start).rjust(4),
          str(z_end).rjust(4),
          end=" ")

    print('Trans: row',
          str(round(row_start,1)).rjust(4),
          str(round(row_end,1)).rjust(4),
          'col',
          str(round(col_start,1)).rjust(4),
          str(round(col_end,1)).rjust(4),
          'z',
          str(round(z_start,1)).rjust(4),
          str(round(z_end,1)).rjust(4),
          end=" ")

    if structure in centers.keys():
        xo,yo,zo = MD589_centers[structure]
        print('COM off by:',
              round(midrow*SCALE-yo, 2),
              round(midcol*SCALE-xo, 2),
              round(midz - zo, 2),
              end=" ")
    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    volume = np.swapaxes(volume, 0, 1)
    volume = np.flip(volume, axis=0)

    try:
        atlasV7_volume[row_start:row_end,col_start:col_end, sec_start:sec_end] += volume
        #atlasV7_volume[y_start:y_end, x_start:x_end, z_start:z_end] += volume
    except ValueError as ve:
        print(ve, end=" ")

    print()


10N_L  : Row  764  852 col 1153 1283 z  195  222 Trans: row  791  879 col 1371 1501 z  195  222 COM off by: 1006.96 223.55 4.01 
10N_R  : Row  764  852 col 1153 1283 z  224  251 Trans: row  789  877 col 1372 1502 z  224  251 COM off by: 1078.02 377.33 1.09 
12N    : Row  776  854 col 1147 1271 z  201  244 Trans: row  803  881 col 1363 1487 z  201  244 
3N_L   : Row  618  659 col  840  903 z  204  230 Trans: row  618  659 col 1077 1140 z  204  230 
3N_R   : Row  618  659 col  840  903 z  217  243 Trans: row  618  659 col 1077 1140 z  217  243 
4N_L   : Row  632  655 col  895  914 z  206  217 Trans: row  622  645 col 1146 1165 z  206  217 COM off by: 159.69 -116.53 -5.5 
4N_R   : Row  632  655 col  895  914 z  230  241 Trans: row  620  643 col 1146 1165 z  230  241 COM off by: 158.13 -106.96 -6.97 
5N_L   : Row  705  791 col  923  996 z  140  170 Trans: row  697  783 col 1173 1246 z  140  170 COM off by: -371.23 433.7 -5.2 
5N_R   : Row  705  791 col  923  996 z  277  307 Trans: row  687

In [11]:
ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()

A new layer named segmentation_[10000, 10000, 20000] is added to:
http://127.0.0.1:43481/v/79511e4b41b234755850fa3f8ff61d241ab5556e/


Downloading:   0%|          | 0/768 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[1490, 2010, 447], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1490, 2010, 447], dtype=int32)



Downloading:  99%|█████████▉| 762/768 [00:02<00:00, 189.86it/s]

Uploading:   0%|          | 0/192 [00:00<?, ?it/s]

Uploading:  19%|█▉        | 36/192 [00:00<00:00, 329.83it/s]

Uploading:  21%|██        | 40/192 [00:00<00:00, 363.57it/s]

Uploading:  21%|██        | 40/192 [00:00<00:00, 363.57it/s]

Uploading:  21%|██        | 40/192 [00:00<00:00, 363.57it/s]

Uploading:  21%|██        | 40/192 [00:00<00:00, 363.57it/s]

Uploading:  21%|██        | 40/192 [00:00<00:00, 363.57it/s]

Uploading:  35%|███▍      | 67/192 [00:00<00:00, 363.57it/s]

Uploading:  49%|████▉     | 94/192 [00:00<00:00, 494.30it/s]

Uploading:  52%|█████▏    | 100/192 [00:00<00:00, 494.30it/s]

Uploading:  60%|█████▉    | 115/192 [00:00<00:00, 494.30it/s]

Uploading:  73%|███████▎  | 141/192 [00:00<00:00, 494.30it/s]

Uploading:  80%|████████  | 154/192 [00:00<00:00, 494.30it/s]

Uploading:  98%|█████████▊| 189/192 [00:00<00:00, 494.30it/s]

Uploading: 100%|██████████| 192/192 [00:00<00:00, 494.30it/s]

Uploadin

Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]


Uploading: 100%|██████████| 48/48 [00:00<00:00, 213.13it/s]



Uploading: 100%|██████████| 48/48 [00:00<00:00, 185.65it/s]

Tasks:  14%|█▍        | 1/7 [00:04<00:29,  4.85s/it]

Uploading: 100%|██████████| 4/4 [00:00<00:00, 19.77it/s][A


Downloading:   3%|▎         | 25/768 [00:00<00:03, 241.93it/s]

Downloading:   4%|▍         | 30/768 [00:00<00:03, 238.18it/s]

Downloading: 

...

Tasks:  35%|███▌      | 7/20 [00:09<00:17,  1.38s/it]73.27it/s]

Downloading:   0%|          | 0/168 [00:00<?, ?it/s]

Downloading:  39%|███▊      | 65/168 [00:00<00:00, 624.19it/s]

Downloading:  64%|██████▍   | 108/168 [00:00<00:00, 344.17it/s]

Downloading:  71%|███████▏  | 120/168 [00:00<00:00, 335.04it/s]

Downloading:  74%|███████▍  | 125/168 [00:00<00:00, 335.04it/s]

Downloading:  80%|███████▉  | 134/168 [00:00<00:00, 335.04it/s]

Downloading:  84%|████████▍ | 141/168 [00:00<00:00, 335.04it/s]

Downloading:  95%|█████████▌| 160/168 [00:00<00:00, 335.04it/s]

Downloading: 100%|██████████| 168/168 [00:00<00:00, 335.04it/s]

Downloading: 100%|██████████| 168/168 [00:00<00:00, 335.04it/s]

Downloading: 100%|██████████| 168/168 [00:00<00:00, 335.04it/s]

Downloading: 100%|██████████| 168/168 [00:00<00:00, 335.04it/s]

Downloading: 100%|██████████| 168/168 [00:00<00:00, 335.04it/s]

Downloading: 100%|██████████| 168/168 [00:00<00:00, 335.04it/s]

Downloading: 100%|██████████| 168/168

Downloading:  53%|█████▎    | 238/448 [00:01<00:01, 119.40it/s]








Downloading:  56%|█████▌    | 250/448 [00:01<00:01, 119.40it/s]








Downloading:  57%|█████▋    | 256/448 [00:01<00:01, 119.40it/s]








Downloading:  62%|██████▏   | 276/448 [00:01<00:01, 105.32it/s]








Downloading:  62%|██████▏   | 279/448 [00:01<00:01, 96.25it/s] 








Downloading:  64%|██████▎   | 285/448 [00:01<00:01, 96.25it/s]








Downloading:  68%|██████▊   | 305/448 [00:01<00:01, 112.48it/s]








Downloading:  72%|███████▏  | 323/448 [00:01<00:01, 121.18it/s]








Downloading:  77%|███████▋  | 347/448 [00:01<00:00, 141.87it/s]








Downloading:  78%|███████▊  | 350/448 [00:01<00:00, 161.43it/s]








Downloading:  84%|████████▍ | 376/448 [00:01<00:00, 168.12it/s]








Downloading:  88%|████████▊ | 396/448 [00:01<00:00, 155.18it/s]








Downloading:  89%|████████▉ | 400/448 [00:01<00:00, 147.84it/s]








Downloading:  95%|█████████▌| 427/448 [00:02<00:00, 160.37it/s]



Downloading:  77%|███████▋  | 345/448 [00:01<00:00, 207.68it/s]





Downloading:  83%|████████▎ | 370/448 [00:01<00:00, 216.87it/s]





Downloading:  83%|████████▎ | 374/448 [00:01<00:00, 220.70it/s]





Downloading:  84%|████████▎ | 375/448 [00:01<00:00, 220.70it/s]





Downloading:  90%|████████▉ | 403/448 [00:01<00:00, 200.63it/s]





Downloading:  90%|█████████ | 404/448 [00:01<00:00, 200.63it/s]





Downloading:  96%|█████████▋| 432/448 [00:01<00:00, 217.96it/s]





Tasks:  65%|██████▌   | 13/20 [01:12<00:50,  7.19s/it]1.81it/s]










Downloading:   0%|          | 0/448 [00:00<?, ?it/s]










Downloading:   6%|▋         | 28/448 [00:00<00:01, 274.95it/s]










Downloading:   7%|▋         | 33/448 [00:00<00:01, 271.13it/s]










Downloading:   8%|▊         | 38/448 [00:00<00:01, 271.13it/s]










Downloading:   8%|▊         | 38/448 [00:00<00:01, 271.13it/s]










Downloading:  14%|█▍        | 63/448 [00:00<00:01, 233.08it/s]










Downloading:  1

Downloading:  65%|██████▍   | 291/448 [00:01<00:00, 159.71it/s]











Downloading:  70%|██████▉   | 313/448 [00:01<00:00, 180.03it/s]











Downloading:  71%|███████   | 316/448 [00:01<00:00, 180.03it/s]











Downloading:  76%|███████▌  | 340/448 [00:01<00:00, 191.83it/s]











Downloading:  82%|████████▏ | 368/448 [00:01<00:00, 210.78it/s]











Downloading:  83%|████████▎ | 370/448 [00:01<00:00, 225.53it/s]











Downloading:  89%|████████▉ | 398/448 [00:01<00:00, 235.61it/s]











Downloading:  89%|████████▉ | 400/448 [00:01<00:00, 251.60it/s]











Downloading:  89%|████████▉ | 400/448 [00:01<00:00, 251.60it/s]











Downloading:  95%|█████████▌| 427/448 [00:01<00:00, 246.65it/s]











Tasks:  80%|████████  | 16/20 [01:40<00:29,  7.30s/it]7.94it/s]

Downloading:   0%|          | 0/224 [00:00<?, ?it/s]

Downloading:  13%|█▎        | 29/224 [00:00<00:00, 289.74it/s]

Downloading:  26%|██▌       | 58/224 [00:00<00:00, 288.83it/s]

Dow

Tasks: 100%|██████████| 999/999 [00:28<00:00, 34.81it/s]
